## DOWNLOADING THE TILE(S)

***

### Providence

In [60]:
import os
import requests
from bs4 import BeautifulSoup

def get_tile_name(lat, lon):
    """
    Returns the USGS tile name formatted correctly (e.g., 'n39w076').
    """
    lat_tile = f"n{int(abs(lat)):02d}" if lat >= 0 else f"s{int(abs(lat)):02d}"
    lon_tile = f"w{int(abs(lon)):03d}" if lon < 0 else f"e{int(abs(lon)):03d}"
    return f"{lat_tile}{lon_tile}"

def get_tiles_in_bbox(min_lat, max_lat, min_lon, max_lon):
    """
    Returns a list of tile names covering a bounding box.
    """
    tile_list = []
    for lat in range(int(min_lat), int(max_lat) + 1):
        for lon in range(int(min_lon), int(max_lon) + 1):
            tile_list.append(get_tile_name(lat, lon))
    return tile_list

def find_latest_tif(tile_name):
    """
    Finds the most recent .tif file in the historical folder for a given tile.
    """
    base_url = f"https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/TIFF/historical/{tile_name}/"
    response = requests.get(base_url)

    if response.status_code != 200:
        print(f"Could not access directory for {tile_name}")
        return None

    soup = BeautifulSoup(response.text, "html.parser")
    for link in soup.find_all("a"):
        href = link.get("href")
        if href and href.endswith(".tif") and f"USGS_13_{tile_name}" in href:
            return base_url + href

    print(f"No .tif file found for {tile_name}")
    return None

tiles = get_tiles_in_bbox(41.664705, 41.932422, -71.569061, -71.198273)
print("Tiles to Download:", tiles)

Tiles to Download: ['n41w071']


### Clip to Providence Boundary

In [61]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd

# File paths
dem_path = "/Users/emmawit/Downloads/USGS_13_n42w072_20240130.tif"
shapefile_path = "/Users/emmawit/Downloads/PVD_Boundary/PVD_Boundary.shp"  # Update this path!
output_path = "/Users/emmawit/Downloads/USGS_13_n42w072_clipped.tif"

# Load shapefile with geopandas
gdf = gpd.read_file(shapefile_path)

# Ensure the shapefile and raster are in the same CRS
with rasterio.open(dem_path) as src:
    if gdf.crs != src.crs:
        gdf = gdf.to_crs(src.crs)

    # Mask the DEM with the shapefile geometry
    out_image, out_transform = mask(src, gdf.geometry, crop=True)
    out_meta = src.meta.copy()

# Update metadata
out_meta.update({
    "driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

# Save the clipped raster
with rasterio.open(output_path, "w", **out_meta) as dest:
    dest.write(out_image)

print("✅ DEM clipped and saved to:", output_path)

✅ DEM clipped and saved to: /Users/emmawit/Downloads/USGS_13_n42w072_clipped.tif


***

### Boston

In [99]:
import os
import requests
from bs4 import BeautifulSoup

def get_tile_name(lat, lon):
    """
    Returns the USGS tile name formatted correctly (e.g., 'n39w076').
    """
    lat_tile = f"n{int(abs(lat)):02d}" if lat >= 0 else f"s{int(abs(lat)):02d}"
    lon_tile = f"w{int(abs(lon)):03d}" if lon < 0 else f"e{int(abs(lon)):03d}"
    return f"{lat_tile}{lon_tile}"

def get_tiles_in_bbox(min_lat, max_lat, min_lon, max_lon):
    """
    Returns a list of tile names covering a bounding box.
    """
    tile_list = []
    for lat in range(int(min_lat), int(max_lat) + 1):
        for lon in range(int(min_lon), int(max_lon) + 1):
            tile_list.append(get_tile_name(lat, lon))
    return tile_list

def find_latest_tif(tile_name):
    """
    Finds the most recent .tif file in the historical folder for a given tile.
    """
    base_url = f"https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/TIFF/historical/{tile_name}/"
    response = requests.get(base_url)

    if response.status_code != 200:
        print(f"Could not access directory for {tile_name}")
        return None

    soup = BeautifulSoup(response.text, "html.parser")
    for link in soup.find_all("a"):
        href = link.get("href")
        if href and href.endswith(".tif") and f"USGS_13_{tile_name}" in href:
            return base_url + href

    print(f"No .tif file found for {tile_name}")
    return None

tiles = get_tiles_in_bbox(42.237964, 42.886589, -73.508142, -69.928393)
print("Tiles to Download:", tiles)

Tiles to Download: ['n42w073', 'n42w072', 'n42w071', 'n42w070', 'n42w069']


### Mosaic

In [100]:
#Downloaded tiles directly from USGS after figuring out which fit ^^

from rasterio.merge import merge

inputfolder= '/Users/emmawit/Downloads/boston_dem'
outfile = 'mosaiced-boston.tif'

tiflist = []

for file in os.listdir(inputfolder):    
    if file.endswith('.tif'):
        tiffile = os.path.join(inputfolder, file)
        tiflist.append(tiffile)


src_files_to_mosaic = []
for fp in tiflist:
    try:
        tile_src = rio.open(fp)
        tile_bounds = tile_src.bounds
        
        src_files_to_mosaic.append(tile_src)
    except:
        continue

print('The number of mosaiced tiles is:', len(src_files_to_mosaic))

mosaic, out_trans = merge(src_files_to_mosaic) 
print('You have mosaiced the results')

out_meta = tile_src.meta.copy()
out_meta.update({"driver": "GTiff",
                  "height": mosaic.shape[1],
                  "width": mosaic.shape[2],
                  "transform": out_trans, 
                  "compress": 'lzw',
                  'BIGTIFF': 'YES'
                  }
               )

with rio.open(outfile, "w", **out_meta) as dest:
     dest.write(mosaic)

The number of mosaiced tiles is: 2
You have mosaiced the results


### Clip to Boston Boundary

In [416]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd

dem_path = "/Users/emmawit/Downloads/mosaiced-boston.tif"
shapefile_path = "/Users/emmawit/Downloads/city_of_boston_outline_boundary_water_included/City_of_Boston_Outline_Boundary_Water_Included.shp"  # Update this path!
output_path = "/Users/emmawit/Downloads/mosaiced-boston-clipped2.tif"

gdf = gpd.read_file(shapefile_path)

with rasterio.open(dem_path) as src:
    if gdf.crs != src.crs:
        gdf = gdf.to_crs(src.crs)

    out_image, out_transform = mask(src, gdf.geometry, crop=True)
    out_meta = src.meta.copy()

out_meta.update({
    "driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

with rasterio.open(output_path, "w", **out_meta) as dest:
    dest.write(out_image)

print("✅ DEM clipped and saved to:", output_path)

✅ DEM clipped and saved to: /Users/emmawit/Downloads/mosaiced-boston-clipped2.tif
